IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\FAULT_M1_5


VOLO M1 FAULT 10%

In [3]:
rcou_m1_fault5 = pd.read_csv("RCOU.csv")
rcou_m1_fault5 = rcou_m1_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m1_fault5 = rcou_m1_fault5[((rcou_m1_fault5['C9'] >= 1350) & (rcou_m1_fault5['C10'] >= 1350) & (rcou_m1_fault5['C11'] >= 1350) & (rcou_m1_fault5['C12'] >= 1350) & (rcou_m1_fault5['C13'] >= 1350) & (rcou_m1_fault5['C14']>= 1350))]
display(rcou_m1_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
6491,192480740,1373,1423,1430,1365,1401,1395
6497,192545491,1426,1445,1478,1390,1467,1403
6500,192576646,1355,1538,1509,1394,1416,1490
6501,192587140,1412,1504,1504,1412,1456,1465
6502,192597516,1523,1403,1461,1472,1514,1414
...,...,...,...,...,...,...,...
14822,284579056,1623,1404,1545,1500,1468,1573
14823,284589530,1586,1441,1553,1480,1440,1586
14824,284599918,1557,1458,1561,1453,1425,1583
14825,284612550,1560,1427,1576,1407,1464,1529


In [4]:
xkf1_m1_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault5 = xkf1_m1_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault5 = xkf1_m1_fault5[((xkf1_m1_fault5['TimeUS'] >= 192480740) & (xkf1_m1_fault5['TimeUS'] <= 284622968	))]
xkf1_m1_fault5 = xkf1_m1_fault5.reset_index(drop=True)
print(xkf1_m1_fault5)

         TimeUS  Roll  Pitch    Yaw
0     192488698  1.25   0.19  19.58
1     192499420  1.34   0.27  19.60
2     192510040  1.42   0.24  19.58
3     192521148  1.48   0.17  19.57
4     192532036  1.47   0.11  19.56
...         ...   ...    ...    ...
8332  284576961  1.35   0.18  19.19
8333  284586972  1.39   0.11  19.12
8334  284597792  1.44   0.02  19.05
8335  284610260  1.63  -0.17  18.93
8336  284620781  1.75  -0.32  18.85

[8337 rows x 4 columns]


In [5]:
att_m1_fault5 = pd.read_csv("ATT.csv")
att_m1_fault5 = att_m1_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault5 = att_m1_fault5[((att_m1_fault5['TimeUS'] >= 192480740) & (att_m1_fault5['TimeUS'] <= 284622968	))]
att_m1_fault5=att_m1_fault5.reset_index(drop=True)
print(att_m1_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737    -0.09  1.47      0.00   0.11   19.50  19.56
...         ...      ...   ...       ...    ...     ...    ...
8337  284578343     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


In [6]:
if len(att_m1_fault5)>len(xkf1_m1_fault5):
    to_add=att_m1_fault5[len(xkf1_m1_fault5):]
    att_m1_fault5=att_m1_fault5[:len(xkf1_m1_fault5)]

if len(xkf1_m1_fault5)>len(att_m1_fault5):
    to_add=xkf1_m1_fault5[len(att_m1_fault5):]

for idx,i in enumerate(att_m1_fault5['Roll']):
    if(xkf1_m1_fault5['Roll'][idx] != i):
        att_m1_fault5['Roll'][idx] = ((att_m1_fault5['Roll'][idx] + xkf1_m1_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Pitch']):
    if(xkf1_m1_fault5['Pitch'][idx] != i):
        att_m1_fault5['Pitch'][idx] = ((att_m1_fault5['Pitch'][idx] + xkf1_m1_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Yaw']):
    if(xkf1_m1_fault5['Yaw'][idx] != i):
        att_m1_fault5['Yaw'][idx] = ((att_m1_fault5['Yaw'][idx] + xkf1_m1_fault5['Yaw'][idx])/2)

att_m1_fault5=pd.concat([att_m1_fault5,to_add])

print(att_m1_fault5)

C:\Users\chiar\AppData\Local\Temp\ipykernel_22872\3051079878.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault5['Roll'][idx] = ((att_m1_fault5['Roll'][idx] + xkf1_m1_fault5['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_22872\3051079878.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault5['Pitch'][idx] = ((att_m1_fault5['Pitch'][idx] + xkf1_m1_fault5['Pitch'][idx])/2)


         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737    -0.09  1.47      0.00   0.11   19.50  19.56
...         ...      ...   ...       ...    ...     ...    ...
8337  284578343     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


C:\Users\chiar\AppData\Local\Temp\ipykernel_22872\3051079878.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault5['Yaw'][idx] = ((att_m1_fault5['Yaw'][idx] + xkf1_m1_fault5['Yaw'][idx])/2)


In [7]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault5 = esc_0_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault5 = esc_0_m1_fault5[((esc_0_m1_fault5['TimeUS'] >= 192480740) & (esc_0_m1_fault5['TimeUS'] <= 284622968))]
esc_0_m1_fault5=esc_0_m1_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault5 = esc_1_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault5 = esc_1_m1_fault5[((esc_1_m1_fault5['TimeUS'] >= 192480740) & (esc_1_m1_fault5['TimeUS'] <= 284622968))]
esc_1_m1_fault5=esc_1_m1_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault5 = esc_2_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault5 = esc_2_m1_fault5[((esc_2_m1_fault5['TimeUS'] >= 192480740) & (esc_2_m1_fault5['TimeUS'] <= 284622968))]
esc_2_m1_fault5=esc_2_m1_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault5 = esc_3_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault5 = esc_3_m1_fault5[((esc_3_m1_fault5['TimeUS'] >= 192480740) & (esc_3_m1_fault5['TimeUS'] <= 284622968))]
esc_3_m1_fault5=esc_3_m1_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault5 = esc_4_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault5 = esc_4_m1_fault5[((esc_4_m1_fault5['TimeUS'] >= 192480740) & (esc_4_m1_fault5['TimeUS'] <= 284622968))]
esc_4_m1_fault5=esc_4_m1_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault5 = esc_5_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault5 = esc_5_m1_fault5[((esc_5_m1_fault5['TimeUS'] >= 192480740) & (esc_5_m1_fault5['TimeUS'] <= 284622968))]
esc_5_m1_fault5=esc_5_m1_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [8]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault5 = imu_0_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault5 = imu_0_m1_fault5[((imu_0_m1_fault5['TimeUS'] >= 192480740) & (imu_0_m1_fault5['TimeUS'] <= 284622968))]

imu_1_m1_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault5 = imu_1_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault5 = imu_1_m1_fault5[((imu_1_m1_fault5['TimeUS'] >= 192480740) & (imu_1_m1_fault5['TimeUS'] <= 284622968))]

imu_2_m1_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault5 = imu_2_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault5 = imu_2_m1_fault5[((imu_2_m1_fault5['TimeUS'] >= 192480740) & (imu_2_m1_fault5['TimeUS'] <= 284622968))]

imu_m1_fault5 = pd.concat((imu_0_m1_fault5, imu_1_m1_fault5, imu_2_m1_fault5))
imu_m1_fault5=imu_m1_fault5.groupby(imu_m1_fault5.TimeUS, as_index=False).mean()

display(imu_m1_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,192482549,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212
1,192485022,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567
2,192488007,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817
3,192490421,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370
4,192492994,-0.168712,-0.092433,-0.014593,0.184634,-0.077262,-9.611569
...,...,...,...,...,...,...,...
31686,284611544,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003
31687,284614408,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220
31688,284617009,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360
31689,284619773,0.226640,-0.287556,-0.125083,-0.290287,-0.079025,-12.356240


In [9]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault5 = pd.merge_ordered(imu_m1_fault5,att_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_0_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_1_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_2_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_3_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_4_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_5_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")
m1_fault5 = pd.merge_ordered(m1_fault5, rcou_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")

In [10]:
index_to_remove=[]
for ind in att_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m1_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

8342
9110
9880
10669
11450
12224
12986


In [11]:
m1_fault5 = m1_fault5[~m1_fault5.TimeUS.isin(index_to_remove)]
m1_fault5["TimeUS"] = m1_fault5["TimeUS"] - m1_fault5.iloc[0]["TimeUS"]
m1_fault5["TimeUS"] = m1_fault5["TimeUS"].astype(int)
m1_fault5["Guasto"] = 1

In [12]:
from datetime import timedelta
m1_fault5=m1_fault5.reset_index(drop=True)
m1_fault5['TimeUS'] = pd.to_datetime(m1_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1
1,00:00:00.002473,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1
2,00:00:00.005458,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1
3,00:00:00.007872,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1
4,00:00:00.010445,-0.168712,-0.092433,-0.014593,0.184634,-0.077262,-9.611569,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31686,00:01:32.128995,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003,4.67,1.44,-1.93,...,2.18,5743.0,3.56,1557.0,1458.0,1561.0,1453.0,1425.0,1583.0,1
31687,00:01:32.131859,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1
31688,00:01:32.134460,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1
31689,00:01:32.137224,0.226640,-0.287556,-0.125083,-0.290287,-0.079025,-12.356240,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1


## Analisi nel tempo e in frequenza

In [13]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [14]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault5)/350)):
        V1=m1_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [15]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = 1
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_22872\183104547.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_22872\183104547.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.017432,0.010058,2.509823,0.101651,0.0,92.0,5.851112,3.315187,0.001027,...,4974.177579,3.430427,0.552766,-0.416075,3.509847,134.0,135.0,15.565676,12.707022,1
1,1.0,-0.017438,0.010029,2.525441,0.101512,0.0,70.0,5.808309,3.760844,0.000684,...,4993.761967,3.430597,0.551201,-0.408345,3.509790,133.0,134.0,15.996884,13.009910,1
2,2.0,-0.017613,0.010011,2.529956,0.101456,0.0,67.0,5.705708,4.180424,0.000346,...,4956.261998,3.430765,0.549645,-0.400614,3.509734,138.0,139.0,16.408625,13.306231,1
3,3.0,-0.017665,0.009984,2.544596,0.101330,0.0,88.0,5.645527,3.218135,0.000098,...,5085.117227,3.430932,0.548098,-0.392884,3.509679,135.0,136.0,16.800342,13.595934,1
4,4.0,-0.017447,0.009973,2.541785,0.101237,0.0,76.0,5.680803,3.203956,-0.000101,...,5464.623485,3.431099,0.546560,-0.385153,3.509624,130.0,131.0,17.171515,13.878968,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85.0,-0.034455,0.009954,1.969268,0.105445,97.0,123.0,3.467092,2.976959,-0.008902,...,8873.232524,3.398922,0.459616,-0.062821,3.465723,153.0,154.0,25.923815,10.311536,1
86,86.0,-0.034619,0.009943,1.971260,0.105447,86.0,87.0,3.509528,3.300953,-0.009052,...,8927.586066,3.399130,0.458581,-0.055857,3.465778,149.0,150.0,25.908144,10.445808,1
87,87.0,-0.034782,0.009932,1.973268,0.105448,90.0,81.0,3.514216,3.402217,-0.008889,...,8970.849719,3.399338,0.457550,-0.048892,3.465834,147.0,148.0,25.837089,10.573015,1
88,88.0,-0.035102,0.009954,1.952177,0.105659,118.0,94.0,3.533276,3.220301,-0.008789,...,9003.672876,3.399544,0.456524,-0.041926,3.465889,149.0,150.0,25.712518,10.692925,1


In [16]:
path_file = path_file.replace(r"M1\FAULT_M1_5", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
